Próximo passo: Baixa no almoxarifado
1. Imprimir nova planilha de ordens do SAP do dia que está sendo reportado

In [7]:
### Importar bibliotecas necessárias ###
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import time
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pyautogui as pag

In [8]:
### Carregar a planilha que foi extraída do Excel com os números das notas
planilhaDf_ordens = pd.read_excel('ordensAbertas.xlsx')

In [9]:
### Avaliar se leu o arquivo corretamente
planilhaDf_ordens.head()

,Data da nota,Sala,Ordem,Nota,Equipamento,Descrição,Resp
0,28.02.2022,A010,321001331155,2688763,159719,Abastecimento de Oleo SSM20,9013
1,28.02.2022,A020,321001331156,2688764,159492,Abastecimento de Oleo SSM20,9013
2,28.02.2022,A020,321001331157,2688765,159491,Abastecimento de Oleo SSM20,9013
3,28.02.2022,C010,321001331158,2688766,159510,Abastecimento de Oleo SSM20,9013
4,28.02.2022,C010,321001331159,2688767,159509,Abastecimento de Oleo SSM20,9013


In [10]:
### Processo de Login com SAP WebGUI ###

## Abre janela para digitar usuário e senha
usuario = pag.prompt('Digita seu usuário')
senha = pag.prompt('Digita sua senha')

## criar o navegador com o webdriver do Chrome
navegador = webdriver.Chrome()
navegador.maximize_window()

## Link de acesso do WebGUI do SAP da ZF
navegador.get("https://swdlp1.zf-world.com/sap/bc/gui/sap/its/webgui")

## Preencher as informações de Login e senha
navegador.find_element(By.XPATH, '//*[@id="sap-user"]').send_keys(usuario)
navegador.find_element(By.XPATH, '//*[@id="sap-password"]').send_keys(senha)

## Clicar no botão para realizar o LOGIN
navegador.find_element(By.XPATH, '//*[@id="LOGON_BUTTON"]').click()

In [11]:
time.sleep(10)
navegador.switch_to.frame("ITSFRAME1");
navegador.find_element(By.XPATH, '//*[@title="Entrar código de transação"]').send_keys("mb1a")
navegador.find_element(By.XPATH, '//*[@title="Entrar código de transação"]').send_keys(Keys.ENTER)
time.sleep(2)

In [12]:
pos = 0
tamanhoPlaniha = len(planilhaDf_ordens)
wait = WebDriverWait(navegador, 10)

for i in planilhaDf_ordens.Ordem:
    time.sleep(1)
    ## Aguardar a tela carregar e limpa os campos que serão digitados
    fastrack = WebDriverWait(navegador, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="M0:46:::6:18"]')))
    fastrack.clear()
    navegador.find_element(By.XPATH, '//*[@id="M0:46:::7:18"]').clear()
    navegador.find_element(By.XPATH, '//*[@id="M0:46:::8:18"]').clear()
    time.sleep(1)
    
    ## Preenche os campos padrões de apontamento de ordem
    navegador.find_element(By.XPATH, '//*[@id="M0:46:::6:18"]').send_keys('261')
    navegador.find_element(By.XPATH, '//*[@id="M0:46:::7:18"]').send_keys('5801')
    navegador.find_element(By.XPATH, '//*[@id="M0:46:::8:18"]').send_keys('5300')
    time.sleep(1)
    navegador.find_element(By.XPATH, '//*[@id="M0:46:::8:18"]').send_keys(Keys.ENTER)
    time.sleep(1)
   
    ## Aguarda a tela carregar e digitar o tempo da execução da atividade, o manutentor e o texto que reabasteceu oleo
    fastrack2 = WebDriverWait(navegador, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="M0:46:2::0:16"]')))
    fastrack2.send_keys(str(int(planilhaDf_ordens['Ordem'][pos])))
    navegador.find_element(By.XPATH, '//*[@id="M0:46:2::0:16"]').send_keys(Keys.ENTER)
    time.sleep(3)
    
    ## Clicar no botão "Por Ordem"
    navegador.find_element(By.XPATH, '//*[@id="M0:37::btn[13]"]').click()
    time.sleep(2)
    
    ## Digitar novamente o numero da ordem e pressionar ENTER
    pag.write(str(int(planilhaDf_ordens['Ordem'][pos])))
    pag.press('enter')
    time.sleep(1)
    
    ## Digitar na caixa lote o lote padrão
    navegador.find_element(By.XPATH, '//*[@id="M0:46:3::0:53"]').send_keys('MRO_NEW')
    time.sleep(1)
    navegador.find_element(By.XPATH, '//*[@id="M0:37::btn[5]"]').click()
    time.sleep(2)
    
    ## Pressionar BOTÃO CTRL + S para salvar
    navegador.find_element(By.XPATH, '//*[@id="M0:36::btn[11]"]').click()
    time.sleep(3)

    pos = pos + 1
    if pos > tamanhoPlaniha:
        pag.alert("Finalizado abertura das notas!")